In [3]:
import numpy as np
import pandas as pd

# Define the KNN class with probability output
class KNN:
    def __init__(self, k=3, distance_metric='euclidean', weighted=False):
        self.k = k
        self.distance_metric = distance_metric
        self.weighted = weighted  # Use weighted voting if True

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict_proba(self, X):
        proba = []
        for x in X:
            distances = self.compute_distance(self.X_train, x)
            k_indices = distances.argsort()[:self.k]
            k_nearest_labels = self.y_train[k_indices]
            k_nearest_distances = distances[k_indices]
            if self.weighted:
                # Weighted probabilities
                weights = 1 / (k_nearest_distances + 1e-5)  # Avoid division by zero
                prob = np.sum(weights * k_nearest_labels) / np.sum(weights)
            else:
                # Simple average
                prob = np.mean(k_nearest_labels)
            proba.append(prob)
        return np.array(proba)

    def predict(self, X, threshold=0.5):
        proba = self.predict_proba(X)
        return (proba >= threshold).astype(int)

    def compute_distance(self, X, x):
        if self.distance_metric == 'euclidean':
            distances = np.sqrt(np.sum((X - x) ** 2, axis=1))
        elif self.distance_metric == 'manhattan':
            distances = np.sum(np.abs(X - x), axis=1)
        else:
            raise ValueError("Unsupported distance metric")
        return distances

# Function to compute ROC AUC manually
def compute_roc_auc(y_true, y_scores, num_thresholds=100):
    thresholds = np.linspace(0, 1, num_thresholds)
    tpr_list = []
    fpr_list = []
    for thresh in thresholds:
        y_pred = (y_scores >= thresh).astype(int)
        tp = np.sum((y_true == 1) & (y_pred == 1))
        tn = np.sum((y_true == 0) & (y_pred == 0))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        fn = np.sum((y_true == 1) & (y_pred == 0))

        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0  # Sensitivity
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0  # 1 - Specificity
        tpr_list.append(tpr)
        fpr_list.append(fpr)
    # Sort false positive rates and true positive rates
    fpr_list, tpr_list = zip(*sorted(zip(fpr_list, tpr_list)))
    # Compute AUC using the trapezoidal rule
    roc_auc = np.trapz(tpr_list, fpr_list)
    return roc_auc

# Define data preprocessing function
def preprocess_data(train_path, test_path):
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)

    # Combine train and test data for consistent preprocessing
    data = pd.concat([train_data, test_data], sort=False).reset_index(drop=True)

    # Drop unnecessary columns
    data.drop(['CustomerId', 'Surname'], axis=1, inplace=True)

    # Handle missing values in 'Gender' and 'Geography'
    data['Gender'] = data['Gender'].fillna('Unknown')
    data['Geography'] = data['Geography'].fillna('Unknown')

    # Map 'Gender' values to 0 and 1; handle unknowns
    gender_mapping = {'Male': 0, 'Female': 1, 'Unknown': -1}
    data['Gender'] = data['Gender'].map(gender_mapping)

    # One-hot encode 'Geography' including 'Unknown'
    geography_dummies = pd.get_dummies(data['Geography'], prefix='Geography')
    data = pd.concat([data, geography_dummies], axis=1)
    data.drop('Geography', axis=1, inplace=True)

    # Convert geography dummy columns from bool to int
    geography_columns = geography_dummies.columns.tolist()
    data[geography_columns] = data[geography_columns].astype(int)

    # Convert numeric columns to numeric data types
    numeric_cols = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
                    'EstimatedSalary']
    for col in numeric_cols:
        if col in data.columns:
            data[col] = pd.to_numeric(data[col], errors='coerce')

    # Handle missing values in numeric columns
    data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].mean())

    # Convert boolean columns to integers
    boolean_cols = ['HasCrCard', 'IsActiveMember']
    for col in boolean_cols:
        if col in data.columns:
            data[col] = data[col].astype(int)

    # Ensure 'Exited' is integer type
    if 'Exited' in data.columns:
        data['Exited'] = data['Exited'].fillna(0).astype(int)

    # Prepare features to scale: select all numeric columns except 'id' and 'Exited'
    features_to_scale = data.select_dtypes(include=[np.number]).columns.tolist()
    features_to_scale = [col for col in features_to_scale if col not in ['Exited', 'id']]

    # Feature scaling using Min-Max scaling
    for feature in features_to_scale:
        min_val = data[feature].min()
        max_val = data[feature].max()
        if max_val - min_val != 0:
            data[feature] = (data[feature] - min_val) / (max_val - min_val)
        else:
            data[feature] = 0  # If no variation, set the feature to zero

    # Split data back into train and test sets
    train = data[:len(train_data)].reset_index(drop=True)
    test = data[len(train_data):].reset_index(drop=True)

    X_train = train.drop(['Exited', 'id'], axis=1).values.astype(float)
    y_train = train['Exited'].values.astype(int)
    X_test = test.drop(['Exited', 'id'], axis=1).values.astype(float)

    train_ids = train['id'].values if 'id' in train.columns else None
    test_ids = test['id'].values if 'id' in test.columns else None

    return X_train, y_train, X_test, train_ids, test_ids

# Check for class imbalance
def check_class_balance(y):
    unique, counts = np.unique(y, return_counts=True)
    imbalance_ratio = counts.min() / counts.max()
    return imbalance_ratio

# Define cross-validation function with ROC AUC
def cross_validate(X, y, k_list, distance_metrics, weighted_options, n_splits=5):
    results = []
    n_samples = len(X)
    indices = np.arange(n_samples)

    # Stratify the folds based on class labels
    class_indices = {}
    for idx, label in enumerate(y):
        if label not in class_indices:
            class_indices[label] = []
        class_indices[label].append(idx)

    fold_indices = [[] for _ in range(n_splits)]
    for label, idx_list in class_indices.items():
        np.random.shuffle(idx_list)
        folds = np.array_split(idx_list, n_splits)
        for i in range(n_splits):
            fold_indices[i].extend(folds[i])

    for k in k_list:
        for metric in distance_metrics:
            for weighted in weighted_options:
                scores = {'accuracy': [], 'precision': [], 'recall': [], 'f1_score': [], 'roc_auc': []}
                for i in range(n_splits):
                    valid_idx = fold_indices[i]
                    train_idx = np.hstack([fold_indices[j] for j in range(n_splits) if j != i])

                    X_train_cv, y_train_cv = X[train_idx], y[train_idx]
                    X_valid_cv, y_valid_cv = X[valid_idx], y[valid_idx]

                    knn = KNN(k=k, distance_metric=metric, weighted=weighted)
                    knn.fit(X_train_cv, y_train_cv)
                    y_proba = knn.predict_proba(X_valid_cv)
                    y_pred = (y_proba >= 0.5).astype(int)

                    # Compute metrics manually
                    tp = np.sum((y_valid_cv == 1) & (y_pred == 1))
                    tn = np.sum((y_valid_cv == 0) & (y_pred == 0))
                    fp = np.sum((y_valid_cv == 0) & (y_pred == 1))
                    fn = np.sum((y_valid_cv == 1) & (y_pred == 0))

                    accuracy = (tp + tn) / (tp + tn + fp + fn)
                    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
                    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
                    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

                    # Compute ROC AUC manually
                    roc_auc = compute_roc_auc(y_valid_cv, y_proba)

                    scores['accuracy'].append(accuracy)
                    scores['precision'].append(precision)
                    scores['recall'].append(recall)
                    scores['f1_score'].append(f1)
                    scores['roc_auc'].append(roc_auc)
                avg_scores = {metric_name: np.mean(scores[metric_name]) for metric_name in scores}
                results.append({
                    'k': k,
                    'distance_metric': metric,
                    'weighted': weighted,
                    **avg_scores
                })
    return pd.DataFrame(results)

# Load and preprocess data
X, y, X_test, train_ids, test_ids = preprocess_data('train.csv', 'test.csv')

# Check class balance
imbalance_ratio = check_class_balance(y)

# Hyperparameter tuning
k_list = [3, 10, 20, 25, 30, 40, 50]
distance_metrics = ['euclidean', 'manhattan']
weighted_options = [True, False]
cv_results = cross_validate(X, y, k_list, distance_metrics, weighted_options, n_splits=5)

# Now sort by 'roc_auc' instead of 'f1_score'
print("\nCross-validation results (sorted by ROC AUC):")
print(cv_results.sort_values(by='roc_auc', ascending=False).head(10))

# Select the best hyperparameters based on highest ROC AUC
best_result = cv_results.sort_values(by='roc_auc', ascending=False).iloc[0]
best_k = best_result['k']
best_metric = best_result['distance_metric']
best_weighted = best_result['weighted']
print(f"\nBest hyperparameters: k={best_k}, distance_metric={best_metric}, weighted={best_weighted}")

# Train on full dataset with optimal hyperparameters and make predictions on test set
knn = KNN(k=int(best_k), distance_metric=best_metric, weighted=best_weighted)
knn.fit(X, y)
test_proba = knn.predict_proba(X_test)

# Save test predictions (probabilities)
submission = pd.DataFrame({
    'id': test_ids,
    'Exited': test_proba  # Use probabilities instead of class labels
})
submission.to_csv('submissions.csv', index=False)



Cross-validation results (sorted by ROC AUC):
     k distance_metric  weighted  accuracy  precision    recall  f1_score  \
18  30       manhattan      True  0.868534   0.763333  0.507422  0.609478   
14  25       manhattan      True  0.869668   0.763243  0.515667  0.615402   
22  40       manhattan      True  0.865934   0.763564  0.488958  0.595872   
26  50       manhattan      True  0.863201   0.762284  0.470823  0.581820   
16  30       euclidean      True  0.866401   0.754448  0.503463  0.603632   
10  20       manhattan      True  0.871001   0.760838  0.528196  0.623411   
8   20       euclidean      True  0.869001   0.751763  0.526220  0.618928   
12  25       euclidean      True  0.865867   0.748679  0.507420  0.604688   
15  25       manhattan     False  0.867601   0.759816  0.505444  0.606904   
19  30       manhattan     False  0.866934   0.748451  0.515662  0.610492   

     roc_auc  
18  0.899394  
14  0.899230  
22  0.898722  
26  0.898211  
16  0.897019  
10  0.897011  
